In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-obp

*Stima dell'utilizzo: 16 minuti su un processore Eagle r3 (NOTA: Questa è solo una stima. Il vostro tempo di esecuzione potrebbe variare.)*

In [ ]:
# This cell is hidden from users;
# it disables linting rules.
# ruff: noqa

## Background


La retropropagazione dell'operatore è una tecnica che consiste nell'assorbire operazioni dalla fine di un circuito quantistico nell'osservabile misurata, riducendo generalmente la profondità del circuito al costo di termini aggiuntivi nell'osservabile. L'obiettivo è retropropagare il più possibile del circuito senza permettere all'osservabile di crescere troppo. Un'implementazione basata su Qiskit è disponibile nell'addon Qiskit OBP, maggiori dettagli possono essere trovati nella corrispondente [documentazione](/guides/qiskit-addons-obp) con un [semplice esempio](/guides/qiskit-addons-obp-get-started) per iniziare.

Considerate un circuito di esempio per il quale deve essere misurata un'osservabile $O = \sum_P c_P P$, dove $P$ sono Pauli e $c_P$ sono coefficienti. Denotiamo il circuito come un singolo unitario $U$ che può essere logicamente partizionato in $U = U_C U_Q$ come mostrato nella figura seguente.

![Circuit diagram showing Uq followed by Uc](../docs/images/tutorials/improving-estimation-of-expectation-values-with-operator-backpropagation/logical-partitioning.avif)

La retropropagazione dell'operatore assorbe l'unitario $U_C$ nell'osservabile facendolo evolvere come $O' = U_C^{\dagger}OU_C = \sum_P c_P U_C^{\dagger}PU_C$. In altre parole, parte del calcolo viene eseguita classicamente tramite l'evoluzione dell'osservabile da $O$ a $O'$. Il problema originale può ora essere riformulato come la misurazione dell'osservabile $O'$ per il nuovo circuito di profondità inferiore il cui unitario è $U_Q$.

L'unitario $U_C$ è rappresentato come un numero di fette $U_C = U_S U_{S-1}...U_2U_1$. Esistono diversi modi per definire una fetta. Per esempio, nel circuito di esempio precedente, ogni strato di $R_{zz}$ e ogni strato di porte $R_x$ possono essere considerati come fette individuali. La retropropagazione comporta il calcolo di $O' = \Pi_{s=1}^S \sum_P c_P U_s^{\dagger} P U_s$ classicamente. Ogni fetta $U_s$ può essere rappresentata come $U_s = exp(\frac{-i\theta_s P_s}{2})$, dove $P_s$ è un Pauli di $n$-qubit e $\theta_s$ è uno scalare. È facile verificare che

$$
U_s^{\dagger} P U_s = P \qquad \text{if} ~[P,P_s] = 0,
$$
$$
U_s^{\dagger} P U_s = \qquad cos(\theta_s)P + i sin(\theta_s)P_sP \qquad \text{if} ~{P,P_s} = 0
$$

Nell'esempio precedente, se ${P,P_s} = 0$, allora dobbiamo eseguire due circuiti quantistici, invece di uno, per calcolare il valore di aspettazione. Pertanto, la retropropagazione può aumentare il numero di termini nell'osservabile, portando a un numero maggiore di esecuzioni del circuito. Un modo per consentire una retropropagazione più profonda nel circuito, evitando che l'operatore cresca troppo, è troncare i termini con coefficienti piccoli, piuttosto che aggiungerli all'operatore. Per esempio, nell'esempio precedente, si può scegliere di troncare il termine che coinvolge $P_sP$ a condizione che $\theta_s$ sia sufficientemente piccolo. Il troncamento dei termini può comportare un minor numero di circuiti quantistici da eseguire, ma farlo comporta un certo errore nel calcolo finale del valore di aspettazione proporzionale all'ampiezza dei coefficienti dei termini troncati.

Questo tutorial implementa un [pattern Qiskit](/guides/intro-to-patterns) per simulare la dinamica quantistica di una catena di spin di Heisenberg utilizzando <a href="https://github.com/Qiskit/qiskit-addon-obp">qiskit-addon-obp</a>.

## Requisiti

Prima di iniziare questo tutorial, assicuratevi di avere installato quanto segue:

- Qiskit SDK v1.2 o versione successiva (`pip install qiskit`)
- Qiskit Runtime v0.28 o versione successiva (`pip install qiskit-ibm-runtime`)
- Addon Qiskit OBP (`pip install qiskit-addon-obp`)
- Addon utils Qiskit (`pip install qiskit-addon-utils`)

## Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import CouplingMap
from qiskit.synthesis import LieTrotter

from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit_addon_utils.slicing import slice_by_gate_types, combine_slices
from qiskit_addon_obp.utils.simplify import OperatorBudget
from qiskit_addon_obp import backpropagate
from qiskit_addon_obp.utils.truncating import setup_budget

from rustworkx.visualization import graphviz_draw

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorV2, EstimatorOptions

## Parte I: Catena di spin di Heisenberg su piccola scala
### Passo 1: Mappare input classici su un problema quantistico
#### Mappare l'evoluzione temporale di un modello quantistico di Heisenberg su un esperimento quantistico.
Il pacchetto [qiskit_addon_utils](https://github.com/Qiskit/qiskit-addon-utils) fornisce alcune funzionalità riutilizzabili per vari scopi.

Il suo modulo [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) fornisce funzioni per generare Hamiltoniane simili a Heisenberg su un dato grafo di connettività.
Questo grafo può essere un [rustworkx.PyGraph](https://www.rustworkx.org/apiref/rustworkx.PyGraph.html) o una [CouplingMap](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.CouplingMap) rendendolo facile da usare nei flussi di lavoro incentrati su Qiskit.

Nel seguito, generiamo una catena lineare `CouplingMap` di 10 qubit.

In [3]:
num_qubits = 10
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/de8ce35e-a2c5-474f-adb9-88c9afb2bd76-0.avif)

Successivamente, generiamo un operatore di Pauli che modella un Hamiltoniana XYZ di Heisenberg.

$$
{\hat{\mathcal{H}}_{XYZ} = \sum_{(j,k)\in E} (J_{x} \sigma_j^{x} \sigma_{k}^{x} + J_{y} \sigma_j^{y} \sigma_{k}^{y} + J_{z} \sigma_j^{z} \sigma_{k}^{z}) + \sum_{j\in V} (h_{x} \sigma_j^{x} + h_{y} \sigma_j^{y} + h_{z} \sigma_j^{z})}
$$

Dove $G(V,E)$ è il grafo della mappa di accoppiamento fornita.

In [4]:
# Get a qubit operator describing the Heisenberg XYZ model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII', 'IIIIIIIIIX', 'IIIIIIIIIY', 'IIIIIIIIIZ', 'IIIIIIIIXI', 'IIIIIIIIYI', 'IIIIIIIIZI', 'IIIIIIIXII', 'IIIIIIIYII', 'IIIIIIIZII', 'IIIIIIXIII', 'IIIIIIYIII', 'IIIIIIZIII', 'IIIIIXIIII', 'IIIIIYIIII', 'IIIIIZIIII', 'IIIIXIIIII', 'IIIIYIIIII', 'IIIIZIIIII', 'IIIXIIIIII', 'IIIYIIIIII', 'IIIZIIIIII', 'IIXIIIIIII', 'IIYIIIIIII', 'IIZIIIIIII', 'IXIIIIIIII', 'IYIIIIIIII', 'IZIIIIIIII', 'XIIIIIIIII', 'YIIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.39269908+0.j, 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j,
 0.78539816+0.j, 1.57079633+0.j, 0.39269908+0.j, 0.78539816+0.j,
 1.57079633+0.j, 0.39269908+0.j, 0.

From the qubit operator, we can generate a quantum circuit which models its time evolution.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a handy function do just that:

In [5]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=2),
)
circuit.draw("mpl", style="iqp", scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif" alt="Output of the previous code cell" />

Dall'operatore di qubit, possiamo generare un circuito quantistico che modella la sua evoluzione temporale.
Ancora una volta, il modulo [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) viene in soccorso con una comoda funzione per fare proprio questo:

In [6]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 18 slices.


![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/1d68f197-ffa4-49de-9fe8-243b1facbd00-0.avif)

### Passo 2: Ottimizzare il problema per l'esecuzione su hardware quantistico
#### Creare fette di circuito da retropropagare
Ricordate, la funzione ``backpropagate`` retropropagherà intere fette di circuito alla volta, quindi la scelta di come affettare può avere un impatto su quanto bene funziona la retropropagazione per un dato problema. Qui, raggrupperemo le porte dello stesso tipo in fette utilizzando la funzione [slice_by_gate_types](https://docs.quantum.ibm.com/api/qiskit-addon-utils/slicing#slice_by_gate_types).

Per una discussione più dettagliata sull'affettamento del circuito, consultate questa [guida pratica](https://qiskit.github.io/qiskit-addon-utils/how_tos/create_circuit_slices.html) del pacchetto [qiskit-addon-utils](https://github.com/Qiskit/qiskit-addon-utils).

In [7]:
op_budget = OperatorBudget(max_qwc_groups=8)

#### Backpropagate slices from the circuit

First we specify the observable to be $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$, $N$ being the number of qubits. We will backpropagate slices from the time-evolution circuit until the terms in the observable can no longer be combined into eight or fewer qubit-wise commuting Pauli groups.

In [8]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits=num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIZII', 'IIIIIIZIII', 'IIIIIZIIII', 'IIIIZIIIII', 'IIIZIIIIII', 'IIZIIIIIII', 'IZIIIIIIII', 'ZIIIIIIIII'],
              coeffs=[0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j, 0.1+0.j,
 0.1+0.j, 0.1+0.j])

#### Vincolare quanto può crescere l'operatore durante la retropropagazione
Durante la retropropagazione, il numero di termini nell'operatore generalmente si avvicinerà rapidamente a $4^N$, dove $N$ è il numero di qubit. Quando due termini nell'operatore non commutano qubit per qubit, abbiamo bisogno di circuiti separati per ottenere i valori di aspettazione corrispondenti ad essi. Per esempio, se abbiamo un'osservabile a 2 qubit $O = 0.1 XX + 0.3 IZ - 0.5 IX$, allora poiché $[XX,IX] = 0$, una misurazione in una singola base è sufficiente per calcolare i valori di aspettazione per questi due termini. Tuttavia, $IZ$ anticommuta con gli altri due termini. Quindi abbiamo bisogno di una misurazione in base separata per calcolare il valore di aspettazione di $IZ$. In altre parole, abbiamo bisogno di due circuiti, invece di uno, per calcolare $\langle O \rangle$. Man mano che il numero di termini nell'operatore aumenta, c'è la possibilità che anche il numero richiesto di esecuzioni del circuito aumenti.

La dimensione dell'operatore può essere limitata specificando il parametro ``operator_budget`` della funzione ``backpropagate``, che accetta un'istanza [OperatorBudget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-simplify#operatorbudget).

Per controllare la quantità di risorse extra (tempo) allocate, limitiamo il numero massimo di gruppi di Pauli commutanti qubit per qubit che l'osservabile retropropagata può avere. Qui specifichiamo che la retropropagazione dovrebbe fermarsi quando il numero di gruppi di Pauli commutanti qubit per qubit nell'operatore supera 8.

In [9]:
# Backpropagate slices onto the observable
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
# Recombine the slices remaining after backpropagation
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 6 slices.
New observable has 60 terms, which can be combined into 6 groups.
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif" alt="Output of the previous code cell" />

#### Retropropagare fette dal circuito
Prima specifichiamo che l'osservabile sia $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$, dove $N$ è il numero di qubit. Retropropagheremo le fette dal circuito di evoluzione temporale finché i termini nell'osservabile non potranno più essere combinati in otto o meno gruppi di Pauli commutanti qubit per qubit.

In [10]:
truncation_error_budget = setup_budget(max_error_per_slice=0.005)

Note that by allocating `5e-3` error per slice for truncation, we are able to remove 1 more slice from the circuit, while remaining within the original budget of eight commuting Pauli groups in the observable. By default, `backpropagate` uses the L1 norm of the truncated coefficients to bound the total error incurred from truncation. For other options refer to the [how-to guide on specifying the p_norm](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

In this particular example where we have backpropagated seven slices, the total truncation error should not exceed ``(5e-3 error/slice) * (7 slices) = 3.5e-2``.
For further discussion on distributing an error budget across your slices, check out [this how-to guide](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [11]:
# Run the same experiment but truncate observable terms with small coefficients
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", scale=0.6)

Backpropagated 7 slices.
New observable has 82 terms, which can be combined into 8 groups.
After truncation, the error in our observable is bounded by 3.266e-02
Note that backpropagating one more slice would result in 114 terms across 12 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/5e8bae1a-ef18-4eb0-9d2a-1ac7bbdced3b-1.avif" alt="Output of the previous code cell" />

Di seguito vedrete che abbiamo retropropagato sei fette, e i termini sono stati combinati in sei e non otto gruppi. Questo implica che retropropagare un'altra fetta causerebbe il superamento di otto del numero di gruppi di Pauli. Possiamo verificare che questo sia il caso ispezionando i metadati restituiti. Notate anche che in questa porzione la trasformazione del circuito è esatta. Cioè, nessun termine della nuova osservabile $O'$ è stato troncato. Il circuito retropropagato e l'operatore retropropagato danno lo stesso risultato esatto del circuito e dell'operatore originale.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

In [13]:
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/65ec9cb1-a4ed-497b-a616-180e9659956f-1.avif)

Successivamente, specificheremo lo stesso problema con gli stessi vincoli sulla dimensione dell'osservabile di output. Tuttavia, questa volta, assegneremo un budget di errore a ciascuna fetta utilizzando la funzione [setup_budget](https://docs.quantum.ibm.com/api/qiskit-addon-obp/utils-truncating#setup_budget). I termini di Pauli con coefficienti piccoli verranno troncati da ciascuna fetta finché il budget di errore non sarà riempito, e il budget residuo verrà aggiunto al budget della fetta successiva. Notate che in questo caso, la trasformazione dovuta alla retropropagazione è approssimata poiché alcuni dei termini nell'operatore sono troncati.

Per abilitare questo troncamento, dobbiamo impostare il nostro budget di errore in questo modo:

In [14]:
pub = (circuit_isa, observable_isa)
bp_pub = (bp_circuit_isa, bp_obs_isa)
bp_trunc_pub = (bp_circuit_trunc_isa, bp_obs_trunc_isa)

Notate che allocando un errore di `5e-3` per fetta per il troncamento, siamo in grado di rimuovere 1 fetta in più dal circuito, rimanendo entro il budget originale di otto gruppi di Pauli commutanti nell'osservabile. Per impostazione predefinita, `backpropagate` utilizza la norma L1 dei coefficienti troncati per limitare l'errore totale derivante dal troncamento. Per altre opzioni fate riferimento alla [guida pratica sulla specifica della p_norm](https://qiskit.github.io/qiskit-addon-obp/how_tos/bound_error_using_p_norm.html).

In questo particolare esempio in cui abbiamo retropropagato sette fette, l'errore totale di troncamento non dovrebbe superare ``(5e-3 errore/fetta) * (7 fette) = 3.5e-2``.
Per ulteriori discussioni sulla distribuzione di un budget di errore tra le vostre fette, consultate [questa guida pratica](https://qiskit.github.io/qiskit-addon-obp/how_tos/truncate_operator_terms.html).

In [15]:
ideal_estimator = Estimator()

# Run the experiments using Estimator primitive to obtain the exact outcome
result_exact = (
    ideal_estimator.run([(circuit, observable)]).result()[0].data.evs.item()
)
print(f"Exact expectation value: {result_exact}")

Exact expectation value: 0.8871244838989416


We shall use <a href="/docs/guides/configure-error-mitigation">resilience_level</a> = 2 for this example.

In [ ]:
options = EstimatorOptions()
options.default_precision = 0.011
options.resilience_level = 2

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run([pub, bp_pub, bp_trunc_pub])

### Step 4: Post-process and return result to desired classical format

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

std_no_bp = job.result()[0].data.stds.item()
std_bp = job.result()[1].data.stds.item()
std_bp_trunc = job.result()[2].data.stds.item()

In [ ]:
print(
    f"Expectation value without backpropagation: {result_no_bp} ± {std_no_bp}"
)
print(f"Backpropagated expectation value: {result_bp} ± {std_bp}")
print(
    f"Backpropagated expectation value with truncation: {result_bp_trunc} ± {std_bp_trunc}"
)

Expectation value without backpropagation: 0.8033194665993642
Backpropagated expectation value: 0.8599808781259016
Backpropagated expectation value with truncation: 0.8868736004169483


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]
stds = [std_no_bp, std_bp, std_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(result_exact)
ax.set_ylim([0.6, 0.92])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif" alt="Output of the previous code cell" />

## Part B: Scale it up!

Let us now use Operator Backpropagation to study the dynamics of the Hamiltonian of a 50-qubit Heisenberg Spin Chain.

### Step 1: Map classical inputs to a quantum problem

We consider a 50-qubit Hamiltonian $\hat{\mathcal{H}}_{XYZ}$ for the scaled up problem with the same values for the $J$ and $h$ coefficients as in the small-scale example. The observable $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ is also the same as before. This problem is beyond classical brute-force simulation.

In [16]:
num_qubits = 50
layout = [(i - 1, i) for i in range(1, num_qubits)]

# Instantiate a CouplingMap object
coupling_map = CouplingMap(layout)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/47cb1ac7-44db-4f96-b49b-e889a920d83c-0.avif" alt="Output of the previous code cell" />

In [17]:
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(np.pi / 8, np.pi / 4, np.pi / 2),
    ext_magnetic_field=(np.pi / 3, np.pi / 6, np.pi / 9),
)
print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIII', 'IIIIIIIIIIII

In [18]:
observable = SparsePauliOp.from_sparse_list(
    [("Z", [i], 1 / num_qubits) for i in range(num_qubits)],
    num_qubits,
)
observable

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIII

### Passo 4: Post-elaborare e restituire il risultato nel formato classico desiderato

In [19]:
circuit = generate_time_evolution_circuit(
    hamiltonian,
    time=0.2,
    synthesis=LieTrotter(reps=4),
)
circuit.draw("mpl", style="iqp", fold=-1, scale=0.6)

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/b10d16cf-95da-42c0-9b47-b2e5a8516c82-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution

In [20]:
slices = slice_by_gate_types(circuit)
print(f"Separated the circuit into {len(slices)} slices.")

Separated the circuit into 36 slices.


We specify the `max_error_per_slice` to be 0.005 as before. However, since the number of slices for this large-scale problem is much higher than the small scale problem, allowing an error of 0.005 per slice may end up creating a large overall backpropagation error. We can bound this by specifying `max_error_total` which bounds the total backpropagation error, and we set its value to 0.03 (which is roughly the same as in the small-scale example).

For this large-scale example, we allow a higher value for the number of commuting groups, and set it to 15.

In [21]:
op_budget = OperatorBudget(max_qwc_groups=15)
truncation_error_budget = setup_budget(
    max_error_total=0.03, max_error_per_slice=0.005
)

Let us first obtain the backpropagated circuit and observable without any truncation.

In [22]:
bp_obs, remaining_slices, metadata = backpropagate(
    observable, slices, operator_budget=op_budget
)
bp_circuit = combine_slices(remaining_slices)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs.paulis)} terms, which can be combined into {len(bp_obs.group_commuting(qubit_wise=True))} groups."
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit.draw("mpl", fold=-1, scale=0.6)

Backpropagated 7 slices.
New observable has 634 terms, which can be combined into 12 groups.
Note that backpropagating one more slice would result in 1246 terms across 27 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/164e2f00-25b6-4cf1-98f8-8b2886f007ee-1.avif" alt="Output of the previous code cell" />

Now allowing for truncation, we obtain:

In [23]:
bp_obs_trunc, remaining_slices_trunc, metadata = backpropagate(
    observable,
    slices,
    operator_budget=op_budget,
    truncation_error_budget=truncation_error_budget,
)

# Recombine the slices remaining after backpropagation
bp_circuit_trunc = combine_slices(
    remaining_slices_trunc, include_barriers=False
)

print(f"Backpropagated {metadata.num_backpropagated_slices} slices.")
print(
    f"New observable has {len(bp_obs_trunc.paulis)} terms, which can be combined into {len(bp_obs_trunc.group_commuting(qubit_wise=True))} groups.\n"
    f"After truncation, the error in our observable is bounded by {metadata.accumulated_error(0):.3e}"
)
print(
    f"Note that backpropagating one more slice would result in {metadata.backpropagation_history[-1].num_paulis[0]} terms "
    f"across {metadata.backpropagation_history[-1].num_qwc_groups} groups."
)
print("The remaining circuit after backpropagation looks as follows:")
bp_circuit_trunc.draw("mpl", fold=-1, scale=0.6)

Backpropagated 10 slices.
New observable has 646 terms, which can be combined into 14 groups.
After truncation, the error in our observable is bounded by 2.998e-02
Note that backpropagating one more slice would result in 1226 terms across 29 groups.
The remaining circuit after backpropagation looks as follows:


<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/c05a85bc-e5ca-4e02-8c96-98b28811f335-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/operator-back-propagation/extracted-outputs/b444d8bc-c800-4aa3-9927-eb807e92194f-1.avif)


## Parte B: Aumentiamo la scala!
Utilizziamo ora la Retropropagazione degli Operatori per studiare le dinamiche dell'Hamiltoniana di una catena di spin di Heisenberg a 50 qubit.
### Passo 1: Mappare gli input classici a un problema quantistico
Consideriamo un'Hamiltoniana a 50 qubit $\hat{\mathcal{H}}_{XYZ}$ per il problema su scala maggiore con gli stessi valori per i coefficienti $J$ e $h$ dell'esempio su piccola scala. Anche l'osservabile $M_Z = \frac{1}{N} \sum_{i=1}^N \langle Z_i \rangle$ è la stessa di prima. Questo problema va oltre la simulazione classica a forza bruta.

In [24]:
# Transpile original experiment
circuit_isa = pm.run(circuit)
observable_isa = observable.apply_layout(circuit_isa.layout)

# Transpile the backpropagated experiment
bp_circuit_isa = pm.run(bp_circuit)
bp_obs_isa = bp_obs_trunc.apply_layout(bp_circuit_isa.layout)

# Transpile the backpropagated experiment with truncated observable terms
bp_circuit_trunc_isa = pm.run(bp_circuit_trunc)
bp_obs_trunc_isa = bp_obs_trunc.apply_layout(bp_circuit_trunc_isa.layout)

In [25]:
print(
    f"2-qubit depth of original circuit: {circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit: {bp_circuit_isa.depth(lambda x:x.operation.num_qubits==2)}"
)
print(
    f"2-qubit depth of backpropagated circuit with truncation: {bp_circuit_trunc_isa.depth(lambda x:x.operation.num_qubits==2)}"
)

2-qubit depth of original circuit: 48
2-qubit depth of backpropagated circuit: 40
2-qubit depth of backpropagated circuit with truncation: 36


### Step 3: Execute using Qiskit primitives

In [26]:
pubs = [
    (circuit_isa, observable_isa),
    (bp_circuit_isa, bp_obs_isa),
    (bp_circuit_trunc_isa, bp_obs_trunc_isa),
]

In [27]:
options = EstimatorOptions()
options.default_precision = 0.01
options.resilience_level = 2
options.resilience.zne.noise_factors = [1, 1.2, 1.4]
options.resilience.zne.extrapolator = ["linear"]

estimator = EstimatorV2(mode=backend, options=options)

In [ ]:
job = estimator.run(pubs)

Per questo problema su scala maggiore abbiamo considerato il tempo di evoluzione come $0.2$ con $4$ passi di Trotter. Il problema è selezionato in modo che vada oltre la simulazione classica a forza bruta, ma possa essere simulato tramite metodo di reti tensoriali. Ciò ci consente di verificare il risultato ottenuto tramite retropropagazione su un computer quantistico con il risultato ideale.

Il valore di aspettazione ideale per questo problema, ottenuto tramite simulazione di reti tensoriali, è $\simeq 0.89$.

In [ ]:
result_no_bp = job.result()[0].data.evs.item()
result_bp = job.result()[1].data.evs.item()
result_bp_trunc = job.result()[2].data.evs.item()

In [ ]:
print(f"Expectation value without backpropagation: {result_no_bp}")
print(f"Backpropagated expectation value: {result_bp}")
print(f"Backpropagated expectation value with truncation: {result_bp_trunc}")

Expectation value without backpropagation: 0.7887194658035515
Backpropagated expectation value: 0.9532818300978584
Backpropagated expectation value with truncation: 0.8913400398926913


In [ ]:
methods = [
    "No backpropagation",
    "Backpropagation",
    "Backpropagation w/ truncation",
]
values = [result_no_bp, result_bp, result_bp_trunc]

ax = plt.gca()
plt.bar(methods, values, color="#a56eff", width=0.4, edgecolor="#8a3ffc")
plt.axhline(0.89)
ax.set_ylim([0.6, 0.98])
plt.text(0.2, 0.895, "Exact result")
ax.set_ylabel(r"$M_Z$", fontsize=12)

Text(0, 0.5, '$M_Z$')

<Image src="../docs/images/tutorials/operator-back-propagation/extracted-outputs/047d448f-aebf-45ff-a81b-83b2d5ca866d-1.avif" alt="Output of the previous code cell" />

Specifichiamo che `max_error_per_slice` sia 0.005 come prima. Tuttavia, poiché il numero di slice per questo problema su larga scala è molto più alto del problema su piccola scala, consentire un errore di 0.005 per slice può finire per creare un grande errore complessivo di retropropagazione. Possiamo limitare questo specificando `max_error_total` che delimita l'errore totale di retropropagazione, e impostiamo il suo valore a 0.03 (che è approssimativamente lo stesso dell'esempio su piccola scala).

Per questo esempio su larga scala, consentiamo un valore più alto per il numero di gruppi commutanti e lo impostiamo a 15.